## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("C:/Users/pzzzz/OneDrive/Documents/Data Analytics/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Thompson,CA,55.7435,-97.8558,34.39,92,100,9.46,overcast clouds
1,1,Provideniya,RU,64.3833,-173.3000,22.10,79,4,6.98,clear sky
2,2,Saskylakh,RU,71.9167,114.0833,5.74,95,100,8.19,overcast clouds
3,3,Cidreira,BR,-30.1811,-50.2056,69.04,89,100,9.17,moderate rain
4,4,Katsuura,JP,35.1333,140.3000,72.19,75,53,7.07,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Atuona,PF,-9.8000,-139.0333,77.36,78,39,19.04,light rain
11,11,Butaritari,KI,3.0707,172.7902,81.46,74,56,11.97,broken clouds
15,15,Pangai,TO,-19.8000,-174.3500,76.59,65,0,0.76,clear sky
16,16,Freeport,US,40.6576,-73.5832,77.58,65,45,5.01,scattered clouds
18,18,Obo,CF,5.3998,26.4869,78.82,82,94,3.36,light rain
21,21,Georgetown,MY,5.4112,100.3354,85.95,79,77,0.00,broken clouds
24,24,Saltpond,GH,5.2091,-1.0606,83.28,72,11,12.41,few clouds
33,33,Biak,ID,-0.9131,122.8766,78.33,85,62,3.04,broken clouds
37,37,Yurimaguas,PE,-5.9000,-76.0833,86.16,43,75,9.22,broken clouds
40,40,Cayenne,GF,4.9333,-52.3333,89.64,70,40,5.01,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isna().any()

City_ID                False
City                   False
Country                 True
Lat                    False
Lng                    False
Max Temp               False
Humidity               False
Cloudiness             False
Wind Speed             False
Current Description    False
dtype: bool

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna(subset = ["Country"], inplace=True)

C:\Users\pzzzz\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Atuona,PF,77.36,light rain,-9.8000,-139.0333,
11,Butaritari,KI,81.46,broken clouds,3.0707,172.7902,
15,Pangai,TO,76.59,clear sky,-19.8000,-174.3500,
16,Freeport,US,77.58,scattered clouds,40.6576,-73.5832,
18,Obo,CF,78.82,light rain,5.3998,26.4869,
21,Georgetown,MY,85.95,broken clouds,5.4112,100.3354,
24,Saltpond,GH,83.28,few clouds,5.2091,-1.0606,
33,Biak,ID,78.33,broken clouds,-0.9131,122.8766,
37,Yurimaguas,PE,86.16,broken clouds,-5.9000,-76.0833,
40,Cayenne,GF,89.64,scattered clouds,4.9333,-52.3333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isna().any()
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Atuona,PF,77.36,light rain,-9.8000,-139.0333,Villa Enata
11,Butaritari,KI,81.46,broken clouds,3.0707,172.7902,Isles Sunset Lodge
15,Pangai,TO,76.59,clear sky,-19.8000,-174.3500,Ha'apai Beach Resort
16,Freeport,US,77.58,scattered clouds,40.6576,-73.5832,The Freeport Inn and Marina
18,Obo,CF,78.82,light rain,5.3998,26.4869,
...,...,...,...,...,...,...,...
660,Jacqueville,CI,85.91,scattered clouds,5.2052,-4.4146,Hotel Le Martin Pecheur
662,Myitkyina,MM,79.30,overcast clouds,25.3833,97.4000,Shwe Phyu Hotel
666,Maragogi,BR,84.09,broken clouds,-9.0122,-35.2225,Salinas do Maragogi All Inclusive Resort
671,Les Cayes,HT,87.71,scattered clouds,18.2000,-73.7500,Villa Orphee


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))